# Capture Video from Camera

Often, we have to capture live stream with a camera. OpenCV provides a very simple interface to do this. Let's capture a video from the camera (I am using the built-in webcam on my laptop), convert it into grayscale video and display it. Just a simple task to get started.

To capture a video, you need to create a **VideoCapture** object. Its argument can be either the device index or the name of a video file. A device index is just the number to specify which camera. Normally one camera will be connected (as in my case). So I simply pass 0 (or -1). You can select the second camera by passing 1 and so on. After that, you can capture frame-by-frame. But at the end, don't forget to release the capture.

## Goal
* Learn to read video, display video, and save video.
* Learn to capture video from a camera and display it.
* You will learn these functions : **`cv.VideoCapture()`**, **`cv.VideoWriter()`**

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import cv2

In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
  print('Can not open the camera')


while True:
  ret, frame = cap.read()
  if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    break

  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BAYER_BG2RGB)
  cv2_imshow(rgb_frame)

  if cv2.waitKey(1) == ord('q'):
    break



cap.release()
cv2.destroyAllWindows()

Can not open the camera
Can't receive frame (stream end?). Exiting ...


**cap.read()** returns a bool **(True/False)**. If the frame is read correctly, it will be True. So you can check for the end of the video by checking this returned value.

Sometimes, cap may not have initialized the capture. In that case, this code shows an error. You can check whether it is initialized or not by the method **cap.isOpened()**. If it is True, OK. Otherwise open it using **cap.open()**.

You can also access some of the features of this video using cap.get(propId) method where propId is a number from 0 to 18. Each number denotes a property of the video (if it is applicable to that video). Full details can be seen here: cv::VideoCapture::get(). Some of these values can be modified using cap.set(propId, value). Value is the new value you want.

For example, I can check the frame width and height by cap.get(cv.CAP_PROP_FRAME_WIDTH) and cap.get(cv.CAP_PROP_FRAME_HEIGHT). It gives me 640x480 by default. But I want to modify it to 320x240. Just use ret = cap.set(cv.CAP_PROP_FRAME_WIDTH,320) and ret = cap.set(cv.CAP_PROP_FRAME_HEIGHT,240).

## Playing Video from file

Playing video from file is the same as capturing it from camera, just change the camera index to a video file name. Also while displaying the frame, use appropriate time for cv.waitKey(). If it is too less, video will be very fast and if it is too high, video will be slow (Well, that is how you can display videos in slow motion). 25 milliseconds will be OK in normal cases.

In [ ]:
cap = cv2.VideoCapture('/content/result_compressed.mp4')

while True :
  ret, frame = cap.read()
  if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    break

  cv2_imshow(frame)

  if cv2.waitKey(1) == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

## Saving a Video

So we capture a video and process it frame-by-frame, and we want to save that video. For images, it is very simple: just use cv.imwrite(). Here, a little more work is required.

This time we create a VideoWriter object. We should specify the output file name (eg: output.avi). Then we should specify the FourCC code (details in next paragraph). Then number of frames per second (fps) and frame size should be passed. And the last one is the isColor flag. If it is True, the encoder expect color frame, otherwise it works with grayscale frame.

FourCC is a 4-byte code used to specify the video codec. The list of available codes can be found in fourcc.org. It is platform dependent. The following codecs work fine for me.



1.   In Fedora: DIVX, XVID, MJPG, X264, WMV1, WMV2. (XVID is more preferable. MJPG results in high size video. X264 gives very small size video)
2.   In Windows: DIVX (More to be tested and added)

3.  In OSX: MJPG (.mp4), DIVX (.avi), X264 (.mkv).

FourCC code is passed as `cv.VideoWriter_fourcc('M','J','P','G')or cv.VideoWriter_fourcc(*'MJPG')` for MJPG.

The below code captures from a camera, flips every frame in the vertical direction, and saves the video.



In [ ]:
import cv2
import numpy as np

# Create a VideoCapture object
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if (cap.isOpened() == False):
    print("Unable to read camera feed")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc(
    'M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

while (True):
    ret, frame = cap.read()

    if ret == True:

        # Write the frame into the file 'output.avi'
        out.write(frame)

        # Display the resulting frame
        cv2.imshow('frame', frame)

        # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Break the loop
    else:
        break

# When everything done, release the video capture and video write objects
cap.release()
out.release()

# Closes all the frames
cv2.destroyAllWindows()


## Calculate the FPS

In [ ]:
import cv2
import numpy as np
import time


cap = cv2.VideoCapture('clip.mp4')
fps = 0

while True:
    start_time = time.time()
    ret, frame = cap.read()

    if not ret:
        print('something wrong..')
        break

    cv2.putText(frame, f'fps: {fps}', (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

    time_taken = time.time() - start_time
    try:
        fps = 1//time_taken
    except ZeroDivisionError as error:
        print(error)


cap.release()
cv2.destroyAllWindows()


## Access Web Camera

In [ ]:
import numpy as np
import cv2 as cv
import time

cap = cv.VideoCapture(0)
fps = 0

while True:
    start_time = time.time()
    ret, frame = cap.read()

    if not ret:
        print('Can\'t access webcam....')

    cv.putText(frame, f'Fps: {fps}', (30, 40),
               cv.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    cv.imshow('Web camera', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    time_taken = time.time() - start_time
    try:
        fps = 1//time_taken
    except ZeroDivisionError as error:
        print(error)

cap.release()
cv.destroyAllWindows()


In [ ]:
# Accessing two or more camera

import numpy as np
import cv2 as cv
import time

cap = cv.VideoCapture(0)
cap1 = cv.VideoCapture(1)
fps = 0

while True:
    start_time = time.time()
    ret, frame = cap.read()
    ret, frame1 = cap1.read()

    if not ret:
        print('Can\'t access webcam....')

    cv.putText(frame, f'Fps: {fps}', (30, 40),
               cv.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
    cv.putText(frame1, f'Fps: {fps}', (30, 40),
               cv.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

    hstack = np.hstack((frame, frame1))
    cv.imshow('Web camera', frame)
    cv.imshow('Web camera', frame1)
    cv.imshow('Two on single wind', hstack)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    time_taken = time.time() - start_time
    try:
        fps = 1//time_taken
    except ZeroDivisionError as error:
        print(error)

cap.release()
cv.destroyAllWindows()
